In [17]:
import pandas as pd
import requests
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [18]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the dataset
file_path = '/Users/artashesmatshkalyan/Downloads/GlobalLandTemperatures_GlobalLandTemperaturesByMajorCity.csv'
df = pd.read_csv(file_path)

# Convert 'dt' column to datetime
df['dt'] = pd.to_datetime(df['dt'])

# Drop rows with missing temperature values
df = df.dropna(subset=['AverageTemperature'])

# Create a new column for year
df['Year'] = df['dt'].dt.year

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard with 2 views: Map View and Best Countries
app.layout = html.Div(style={'backgroundColor': '#2d2d2d', 'color': 'white'}, children=[
    html.H1("Global Temperature Analysis Dashboard", style={'textAlign': 'center', 'color': 'white'}),
    
    # Map view for average temperature by city (with increased size)
    html.Div([
        dcc.Graph(id='map-view', figure=px.scatter_geo(
            df, locations="Country", locationmode="country names",
            color="AverageTemperature", hover_name="City",
            size=df['AverageTemperature'].abs(),  # Use absolute value for size
            animation_frame="Year",
            projection="natural earth",
            title="Global Average Temperature by City",
            template='plotly_dark').update_layout(
                paper_bgcolor='#2d2d2d', geo_bgcolor='#2d2d2d',
                font_color='white', geo=dict(showland=True, landcolor='darkgrey'),
                height=600  # Set larger height for map view
        ))
    ], style={'width': '100%', 'display': 'inline-block'}),
    
    # Scatter plot for best countries for future living based on average temperature
    html.Div([
        dcc.Graph(id='best-countries', style={'height': '600px'}, figure=px.scatter(
            df.groupby('Country')['AverageTemperature'].mean().reset_index(),
            x='AverageTemperature', y='Country', color='AverageTemperature',
            title="Best Countries for Future Living (Moderate Avg Temp)",
            template='plotly_dark').update_traces(marker=dict(size=10))
            .update_layout(
                paper_bgcolor='#2d2d2d', font_color='white',
                title_font_color='white', plot_bgcolor='#2d2d2d',
                height=600  # Increase height for scatter plot
        ))
    ], style={'width': '100%', 'display': 'inline-block'}),
])

# Highlight Canada, Russia, and Chile in the scatter plot
@app.callback(
    Output('best-countries', 'figure'),
    [Input('best-countries', 'hoverData')]
)
def update_best_countries_plot(hoverData):
    best_countries_df = df.groupby('Country')['AverageTemperature'].mean().reset_index()
    
    # Set different sizes and colors for Canada, Russia, and Chile to highlight them
    best_countries_df['MarkerSize'] = 10  # Default size
    best_countries_df.loc[best_countries_df['Country'].isin(['Canada', 'Russia', 'Chile']), 'MarkerSize'] = 20  # Highlight size
    
    fig = px.scatter(
        best_countries_df, x='AverageTemperature', y='Country', color='AverageTemperature',
        title="Best Countries for Future Living (Moderate Avg Temp)", size='MarkerSize',
        template='plotly_dark', labels={'AverageTemperature': 'Average Temp (°C)'}
    )
    
    # Custom styling to make Canada, Russia, and Chile stand out
    fig.update_traces(marker=dict(line=dict(width=2, color='white')))  # Add outline to markers
    fig.update_layout(paper_bgcolor='#2d2d2d', font_color='white', title_font_color='white')
    
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
